In [0]:
df_main = spark.sql("SELECT * from PARQUET.`abfss://silver@salesetldatalake.dfs.core.windows.net/silver_data`")

In [0]:
%sql
SHOW CATALOGS;

In [0]:
df_model = spark.sql("SELECT * FROM sales_catalog.gold.dim_models")
# df_model.display()

In [0]:
df_branch = spark.sql("SELECT * FROM sales_catalog.gold.dim_branch")
df_dealer = spark.sql("SELECT * FROM sales_catalog.gold.dim_dealer")
df_date = spark.sql("SELECT * FROM sales_catalog.gold.dim_date")

In [0]:
fact_df = df_main.join(df_branch,df_main.Branch_ID == df_branch.Branch_ID, how='left')\
    .join(df_dealer,df_main.Dealer_ID == df_dealer.Dealer_ID, how='left')\
        .join(df_model,df_main.Model_ID == df_model.Model_ID, how='left')\
            .join(df_date,df_main.Date_ID == df_date.Date_ID, how='left')\
                .select(df_main.Revenue, df_main.Units_Sold, df_branch.branch_key, df_dealer.dealer_key, df_date.date_key, df_model.Product_Name, df_model.model_key)

In [0]:
fact_df.display()

In [0]:
if spark.catalog.tableExists("fact_df"):
    deltatab = DeltaTable.forName(spark, "sales_catalog.gold.fact_df")
    deltatab.alias("t").merge(fact_df.alias("s"), "t.branch_key = s.branch_key AND t.dealer_key = s.dealer_key AND t.date_key = s.date_key AND t.model_key = s.model_key")\
        .whenMatched().updateAll()\
            .whenNotMatchedInsertAll()\
            .execute()

else:
    fact_df.write.format('delta').mode('overwrite').option("path","abfss://gold@salesetldatalake.dfs.core.windows.net/fact_df")\
        .saveAsTable("sales_catalog.gold.fact_df")

In [0]:
%sql
select * from sales_catalog.gold.fact_df;